In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amazon-ratings/ratings_Beauty.csv
/kaggle/input/chocolate-bar-ratings/flavors_of_cacao.csv
/kaggle/input/amazon-fine-food-reviews/hashes.txt
/kaggle/input/amazon-fine-food-reviews/Reviews.csv
/kaggle/input/amazon-fine-food-reviews/database.sqlite
/kaggle/input/mobile-games-ab-testing-cookie-cats/cookie_cats.csv


In [2]:
# Ölçümleme problemleri başlığı altında inceleyeceğimiz konular: 
# -Ürün puanlama
#   * Ortalama puana göre
#   * Puan tarihine göre
#   * Kullanıcı temelli puanlama
# Ürün sıralama
#   * Derecelendirmeye göre sıralama
#   * Yorum ve Satın alma sayısına göre sıralama
#   * Ağırlıklı ortalama ile sıralama
#   * BAR (Bayesian Average Rating) skoru ile sıralama
# Yorum sıralama
#   * Üst-Alt farkı ile sıralama
#   * Ortalama puan ile sıralama
#   * WLB (Wilson Lower Bound) puanına göre sıralama
# AB Testi
#   * İki örneklem ortalaması karşılaştırma
#   * İki grup oran karşılaştırma
#   * İkiden fazla grupp ortalaması karşılaştırma (ANOVA)

In [3]:
# -Ürün puanlama
#   * Ortalama puana göre
#   * Puan tarihine göre
# İlk iki örnek için veri setimiz: Chocolate Bar Ratings
chocolate = pd.read_csv("../input/chocolate-bar-ratings/flavors_of_cacao.csv")
#   * Kullanıcı temelli puanlama
# Bu örnekte kullanacağımız veri setimiz: Amazon - Ratings (Beauty Products)
amazon = pd.read_csv("../input/amazon-ratings/ratings_Beauty.csv")

In [4]:
#İlk veri setimizde öncelikle düzenleme yapmamız gerekiyor. Sütun isimlerini düzenleyelim: 
chocolate.columns = ["Company", "Specific Bean Origin\nor Bar Name", "REF", "Review_Date", "Cocoa Perc.",
                    "Company Location", "Rating", "Bean Type", "Broad Bean Origin"]
#Veri setimizde kullanacağımız sütunlarla yeni bir DataFrame oluşturalım
chocolate_df = chocolate[["Company", "Review_Date", "Rating"]]

In [5]:
# * Ortalama puana göre sıralama (aynı zamanda ürün sıralamadaki ilk kısmı da içermektedir)
avg_score = chocolate_df.groupby("Company").agg({"Rating": "mean"}).sort_values("Rating",  ascending=False)

In [6]:
# * Puan tarihine göre sıralama: Bu kısımda tarihleri aralıklara bölerek sıralama yapabilir. Bu veri setinde
#tarih formatı yıl bazında gözüküyor. Aşağıdaki gibi bölebiliriz: 
chocolate_df["Review_Date_Interval"] = chocolate_df["Review_Date"].apply(lambda x: "2015_2017" if x > 2014 else
                                                                        ("2012_2014" if x > 2011 else
                                                                        ("2009_2011" if x > 2008 else
                                                                        "<2009")))
#Puanların ilgili aralıklara göre ve çikolata şirketine göre ortalamalarını içerecek şekilde bir gruplama
#yapalım 
date_score = chocolate_df.groupby(["Company", "Review_Date_Interval"])["Rating"].mean().reset_index()

#Tarihe göre ağırlıklı ortalama yaparken, her bir şirkete ait yorumların yukarıda belirtilen tarih aralıkları
#içerisinde en az 1 değerlendirmesi olması gerekir. Aksi taktirde o şirketin puanı olması gereken puandan
#daha düşük olacaktır. Her aralıkta olup olmadığını aşağıdaki gibi kontrol edebiliriz: 
date_interval_count = date_score["Company"].value_counts().reset_index()
date_interval_count .columns = ["Company", "Number_of_Intervals"]
date_score = date_score.merge(date_interval_count )
date_score = date_score.loc[date_score["Number_of_Intervals"] == 4]  #4 aralıkta da değerlendirme olanlar alınır

#şimdi gönül rahatlığıyla ağırlıklı puanları alabiliriz. (qcut, null değerleri çıkarma, ürüne göre 
#değerlendirme, boş olan verileri bir ML modeliyle doldurma - Ör: KNN, Matris faktorizasyonu) 
date_score["Weighted_Rate"] = date_score.apply(lambda x: x["Rating"]*0.33 if x["Review_Date_Interval"] == "2015_2017"
                                     else (x["Rating"]*0.27 if x["Review_Date_Interval"] == "2012_2014" else
                                     (x["Rating"]*0.23 if x["Review_Date_Interval"] == "2009_2011" else
                                      x["Rating"]*0.17)),
                                     axis=1)

#göreceğiniz üzere 4 aralıkta da olan firma sayısı oldukça düşük. eğer daha sağlıklı bir karşılaştırma yapmak
#istiyorsak, aralık sayısını azaltabiliriz. 
date_score_final = date_score.groupby("Company").agg({"Weighted_Rate": "sum"}).sort_values("Weighted_Rate", 
                                                                                           ascending=False)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
date_score_final

,Weighted_Rate
Company,
Valrhona,3.670125
Pacari,3.548750
Domori,3.499667
Bonnat,3.475375
Rogue,3.447708
Guittard,3.323018
TCHO,2.634375


In [8]:
# * Kullanıcı temelli puanlama: Burada da aslında ağırlıklı değerlendirme yaptığımız için yukarıdakine benzer
#şekilde her aralıkta değerlendirme olup olmadığı kontrol edilmelidir. 

#veri setimizin boyutu büyük. Daha hızlı ilerlemek adına veri setimizin içerisinden 100000 satırlık bir 
#örneklem alıp çalışabiliriz. Normalde böyle ilerlemek çok da alışılmış bir durum değil fakat pratiklik 
#açısından bu seferlik böyle yapalım.
amazon_df = amazon.sample(100000)

#Bu çalışmada kullanıcıların değerlendirme sayısına bakacağız. Öncelikle değerlendirme sayısını bulmalıyız:
user_votes = amazon["UserId"].value_counts().reset_index()
#değerlendirme sayılarını ana veri setimize ekleyelim
user_votes.columns = ["UserId", "Vote_Count"]
amazon_df = amazon_df.merge(user_votes)

#değerlendirme sayılarını aralıklara bölelim
amazon_df["UserId_Interval"] = amazon_df["Vote_Count"].apply(lambda x: ">50" if x > 50 else
                                                                        ("21_50" if x > 20 else
                                                                        ("11_20" if x > 10 else
                                                                        "1_10")))

#bir önceki çalışmay benzer şekilde gruplama yapıyoruz. Sonrasında da aralık kontrolü yapıyoruz
user_based_score = amazon_df.groupby(["ProductId", "UserId_Interval"])["Rating"].mean().reset_index()

user_interval_count  = user_based_score["ProductId"].value_counts().reset_index()
user_interval_count.columns = ["ProductId", "Number_of_Intervals"]
user_based_score = user_based_score.merge(user_interval_count)
user_based_score = user_based_score.loc[user_based_score["Number_of_Intervals"] == 4]

#ağırlıklandırma yapalım ve sonuca ulaşalım: 
user_based_score["Weighted_Rate"] = user_based_score.apply(lambda x: x["Rating"]*0.33 if 
                                                           x["UserId_Interval"] == ">50"
                                     else (x["Rating"]*0.27 if x["UserId_Interval"] == "21_50" else
                                     (x["Rating"]*0.23 if x["UserId_Interval"] == "11_20" else
                                      x["Rating"]*0.17)),
                                     axis=1)
user_based_score_final = user_based_score.groupby("ProductId").agg({"Weighted_Rate": "sum"}).sort_values(
    "Weighted_Rate", ascending=False)

In [9]:
user_based_score_final

,Weighted_Rate
ProductId,
B000ZV0D9W,5.000000
B004OE9RO8,5.000000
B009JKDCEI,5.000000
B00AWLBACE,5.000000
B00IP42FBA,4.967143
...,...
B0012J30LY,2.900000
B00AYOHWQW,2.890000
B007RTR8TS,2.743333


In [10]:
# Ürün sıralama
#   * Derecelendirmeye göre sıralama (yukarıda yapıldığı için tekrar değinmeyeceğim)
#   * Yorum ve Satın alma sayısına göre sıralama
#   * Ağırlıklı ortalama ile sıralama
#   * BAR (Bayesian Average Rating) skoru ile sıralama
#Kullanacağımız veri seti:
fine_food = pd.read_csv("../input/amazon-fine-food-reviews/Reviews.csv")

In [11]:
fine_food.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      568454 non-null  int64 
 1   ProductId               568454 non-null  object
 2   UserId                  568454 non-null  object
 3   ProfileName             568438 non-null  object
 4   HelpfulnessNumerator    568454 non-null  int64 
 5   HelpfulnessDenominator  568454 non-null  int64 
 6   Score                   568454 non-null  int64 
 7   Time                    568454 non-null  int64 
 8   Summary                 568427 non-null  object
 9   Text                    568454 non-null  object
dtypes: int64(5), object(5)
memory usage: 43.4+ MB


In [12]:
fine_food.nunique()

Id                        568454
ProductId                  74258
UserId                    256059
ProfileName               218416
HelpfulnessNumerator         231
HelpfulnessDenominator       234
Score                          5
Time                        3168
Summary                   295742
Text                      393579
dtype: int64

In [13]:
#Time değişkeni unix timestamp formatında, tarihe dönüştürmek için; 
fine_food["Time"] = pd.to_datetime(fine_food["Time"],unit='s')
fine_food["Time"].head()

0   2011-04-27
1   2012-09-07
2   2008-08-18
3   2011-06-13
4   2012-10-21
Name: Time, dtype: datetime64[ns]

In [14]:
product_table = fine_food.groupby("ProductId").agg({"Text": "count", "Score": "mean"}).reset_index()

In [15]:
product_table.columns = ["ProductId", "Review_Count", "Avg_Score"]

In [16]:
#bu veri setinde satın alma sayısı yer almıyor. Ben de satın alma sayısı en az yorum sayısı kadar, en fazla 
#yorum sayısının 1.5 katı olacak şekilde rastgele bir sayı veriyorum. Bunu sadece burada örnek olarak
#göstermek amacıyla yapıyorum
import random
product_table["Purchase_Count"] = 0
for row in range(product_table["ProductId"].count()-1):
    product_table["Purchase_Count"][row] = random.randint(product_table["Review_Count"][row], 
                                                          int(product_table["Review_Count"][row]*1.5)+1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [17]:
# *yorum sayısına göre sıralama
product_table.sort_values("Review_Count", ascending=False)

,ProductId,Review_Count,Avg_Score,Purchase_Count
71170,B007JFMH8M,913,4.582694,1347
37898,B0026RQTGE,632,4.588608,671
42257,B002QWHJOU,632,4.588608,878
42263,B002QWP89S,632,4.588608,810
42264,B002QWP8H0,632,4.588608,634
...,...,...,...,...
21719,B000YPQC08,1,5.000000,2
51301,B003YU5T6I,1,5.000000,2
21720,B000YPQC44,1,5.000000,2
21721,B000YPQE6U,1,5.000000,1


In [18]:
# *satın alma sayısına göre sıralama
product_table.sort_values("Purchase_Count", ascending=False)

,ProductId,Review_Count,Avg_Score,Purchase_Count
71170,B007JFMH8M,913,4.582694,1347
42257,B002QWHJOU,632,4.588608,878
71327,B007M83302,564,4.310284,842
35243,B001RVFEP2,564,4.310284,818
42263,B002QWP89S,632,4.588608,810
...,...,...,...,...
63265,B00551E1P4,1,5.000000,1
63266,B005520KKS,1,5.000000,1
63267,B00552CKZG,1,1.000000,1
7369,B000CMI5TY,1,5.000000,1


In [19]:
# *Ağırlıklı ortalama ile sıralama: Puan, yorum ve satın alma sayısını hesaba katacağız. Yalnız bir sorun var!
# Bahsettiğim değişkenler aynı skalada değil, o yüzden ağırlıklandırma yapmak için hepsini puandaki gibi 
#1-5 arası skalaya getirmek gerekiyor. Bunun için MinMaxScaler metodunu kullanacağız. 
from sklearn.preprocessing import MinMaxScaler
#Satın alma sayısını 1-5 arası skalaya çevirme
product_table["Purchase_Count_Scaled"] = MinMaxScaler(feature_range=(1, 5)). \
    fit(product_table[["Purchase_Count"]]). \
    transform(product_table[["Purchase_Count"]])
#Yorum sayısını 1-5 arası skalaya çevirme
product_table["Review_Count_Scaled"] = MinMaxScaler(feature_range=(1, 5)). \
    fit(product_table[["Review_Count"]]). \
    transform(product_table[["Review_Count"]])
#Ağırlıklı puan hesaplama (Yorum: %33, Satın Alma Sayısı: %27, Puan: %40)
product_table["Weighted_Score"] = product_table["Review_Count_Scaled"] * 33 / 100 + \
product_table["Purchase_Count_Scaled"] * 27 / 100 + product_table["Avg_Score"] * 40 / 100
#Ağırlıklı puana göre sıralama
product_table.sort_values("Weighted_Score", ascending=False)

,ProductId,Review_Count,Avg_Score,Purchase_Count,Purchase_Count_Scaled,Review_Count_Scaled,Weighted_Score
71170,B007JFMH8M,913,4.582694,1347,5.000000,5.000000,4.833078
42257,B002QWHJOU,632,4.588608,878,3.607275,3.767544,4.052697
42263,B002QWP89S,632,4.588608,810,3.405345,3.767544,3.998176
46205,B003B3OOPA,623,4.739968,694,3.060876,3.728070,3.952687
16945,B000NMJWZO,542,4.881919,721,3.141054,3.372807,3.913878
...,...,...,...,...,...,...,...
29862,B001EPQ9Y6,1,1.000000,1,1.002970,1.000000,1.000802
66448,B005OKDC9C,1,1.000000,1,1.002970,1.000000,1.000802
31290,B001FPZ324,1,1.000000,1,1.002970,1.000000,1.000802
18644,B000RDGI74,1,1.000000,1,1.002970,1.000000,1.000802


In [20]:
# *Bayesian Average Rating (BAR) skoru ile sıralama: Bunu gerçekleştirebilmek için, her ürünün 1-5 arası 
#puan sayısını çıkartmamız gerekiyor:
fine_food["1_point"] = fine_food["Score"].apply(lambda x: 1 if x==1 else 0)
fine_food["2_point"] = fine_food["Score"].apply(lambda x: 1 if x==2 else 0)
fine_food["3_point"] = fine_food["Score"].apply(lambda x: 1 if x==3 else 0)
fine_food["4_point"] = fine_food["Score"].apply(lambda x: 1 if x==4 else 0)
fine_food["5_point"] = fine_food["Score"].apply(lambda x: 1 if x==5 else 0)

#Şimdi ürüne göre gruplama yapabilir ve her puandan kaç adet olduğunu görebiliriz:
BAR_score_df = fine_food.groupby("ProductId").agg({"1_point": "sum", "2_point": "sum", "3_point": "sum", 
                                                 "4_point": "sum", "5_point": "sum"})

In [21]:
#BAR score için fonksiyonumuzu yazalım: 
import math
import scipy.stats as st
def bayesian_average_rating(n, confidence=0.95):
    if sum(n) == 0:
        return 0
    K = len(n)
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    N = sum(n)
    first_part = 0.0
    second_part = 0.0
    for k, n_k in enumerate(n):
        first_part += (k + 1) * (n[k] + 1) / (N + K)
        second_part += (k + 1) * (k + 1) * (n[k] + 1) / (N + K)
    score = first_part - z * math.sqrt((second_part - first_part * first_part) / (N + K + 1))
    return score
#şimdi de formülümüzü uygulayalım
BAR_score_df["BAR_score"] = BAR_score_df.apply(lambda x: bayesian_average_rating(x[["1_point",
                                                                                    "2_point",
                                                                                    "3_point",
                                                                                    "4_point",
                                                                                    "5_point"]]), axis=1)
#BAR skoruna göre sıralamamızı görelim: 
BAR_score_df.sort_values("BAR_score", ascending=False)

,1_point,2_point,3_point,4_point,5_point,BAR_score
ProductId,,,,,,
B000NMJWZO,3,3,9,25,502,4.820006
B000ED9L9E,0,0,1,1,111,4.794002
B003QDRJXY,3,3,4,14,240,4.721808
B001E8DHPW,4,1,13,32,339,4.714833
B000GAT6NG,4,1,13,32,339,4.714833
...,...,...,...,...,...,...
B004US0VMU,6,0,0,0,0,1.129030
B005QD0V8M,7,0,0,0,0,1.102898
B004OV3VGQ,8,0,0,0,0,1.083048


In [22]:
# Yorum sıralama
#   * Üst-Alt farkı ile sıralama
#   * Ortalama puan ile sıralama
#   * WLB (Wilson Lower Bound) puanına göre sıralama
#Burada, bir önceki konuda kullandığımız veri seti bizim için yeterli. Bu veri seti üzerinden çalışabiliriz.


In [23]:
#   * Üst-Alt farkı ile sıralama. Bu veri setinde, "HelpfulnessNumerator" sütunu Up vote sayısını gösteriyor
# Down vote sayısına ise Up ve Down toplam sayılarından ("HelpfulnessDenominator") Up sayısını çıkartarak
#erişebiliriz. Bu sefer ürünleri değil, yorumları sıralıyoruz, o yüzden groupby yapmamıza gerek yok
fine_food["Up-Down_Sorting"] = fine_food["HelpfulnessNumerator"] - \
(fine_food["HelpfulnessDenominator"] - fine_food["HelpfulnessNumerator"])
#sort_values metodu ile sıralayarak sonuca ulaşabiliriz
fine_food.sort_values("Up-Down_Sorting", ascending=False)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,1_point,2_point,3_point,4_point,5_point,Up-Down_Sorting
190733,190734,B000FI4O90,A1GQGYROVZVW49,P. Schmidt,866,878,5,2006-11-28,Works as Advertised - Classy Product,see update at end of review<br /><br />*******...,0,0,0,0,1,854
566779,566780,B001PQTYN2,A1QB2Y8GSME58Y,Benjamin Chodroff,808,815,5,2009-12-14,sauce not for mortals,I purchased a burrito from a small shop a few ...,0,0,0,0,1,801
207712,207713,B00012182G,A1JUGIQDY6UYSM,"BabbChuck ""BabbChuck""",844,923,3,2009-09-08,Whole Rabbit - NOT!,"I ordered one of these Fresh ""Whole"" Rabbits, ...",0,0,1,0,0,765
235722,235723,B001F10XUU,A39V22BIBUMMB3,"Y. Hughes ""Notmyrealname""",580,593,1,2011-07-02,Lost in Translation: Truth,"This product is called ""Hunmatsu-RyokuCha,"" in...",1,0,0,0,0,567
373575,373576,B009K2BBQG,A1B4MIXYVIKMU2,"D. Truong ""Duke of New Mexico""",559,562,5,2011-09-02,Works great! But how does it compare to Solofi...,This Ecobrew reusable Keurig K-cup is great fo...,0,0,0,0,1,556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451344,451345,B003QDRJXY,AACKAWVXFJ2WT,christine,1,91,1,2011-12-17,My review is based on the comments of your rev...,"I'm looking for a product to cook with, specif...",1,0,0,0,0,-89
533347,533348,B0028667PG,A1SZJQFD8ECPSX,nobody,3,106,1,2011-10-13,this noodle sucks,I just burned my finger while i was trying to ...,1,0,0,0,0,-100
557662,557663,B004YV7RK2,A19VVEHTYQT3DM,"S. Curtis ""Peek lover""",13,135,2,2011-11-21,Tooo much money,You can get a 5 pound Hershey bar at KMART for...,0,1,0,0,0,-109
231087,231088,B00315HJ8C,A1XFF11MZPQ04T,berushworthy,9,133,1,2012-02-16,Lies,The world's largest gummy bear is in Austin. I...,1,0,0,0,0,-115


In [24]:
#   * Ortalama puan ile sıralama: Up / (Up+Down) formülünü uygulayabiliriz. Bu veri setimizde zaten Up+Down
# için bir değişkenimiz var (HelpfulnessDenominator). Yalnız, hiç up-down olmayan yorumlarda hata almamak
#adına, bu değerleri 0'a eşitlemek gerekecektir.

fine_food["Avg_Review_Point"] = fine_food.apply(lambda x: 
                                                x["HelpfulnessNumerator"] / x["HelpfulnessDenominator"] if 
                                                x["HelpfulnessDenominator"] != 0 else 0, axis=1)
# Ortalama puana göre sıralıyoruz. Burada 1'den büyük değer olmaması gerekiyor fakat 2 satırın değeri 1'den 
#büyük. Veri setinde yapılmış bir hata fakat bunları göz ardı edebiliriz. 
fine_food.sort_values("Avg_Review_Point", ascending=False)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,1_point,2_point,3_point,4_point,5_point,Up-Down_Sorting,Avg_Review_Point
64421,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,2008-10-25,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...,0,0,0,0,1,5,3.0
44736,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,2008-06-08,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...,0,0,0,1,0,4,1.5
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,2011-04-27,Good Quality Dog Food,I have bought several of the Vitality canned d...,0,0,0,0,1,1,1.0
264020,264021,B004EAGP74,A1HDWGY5KWY11Y,Brooklynite,2,2,5,2008-01-07,Best Deal Ever!,You are not going to get a better deal on any ...,0,0,0,0,1,2,1.0
497457,497458,B006FFC6E2,AGPCMCQ9NX54Y,Horner Sales Inc.,2,2,5,2012-02-19,Amazing Sauces!,The American Stockyard line is handcrafted usi...,0,0,0,0,1,2,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246685,246686,B007M8FRU6,A1W5HSL3YBS265,Lucky Lisa,0,0,5,2012-10-17,Carmel Apple Tea,"very tasty, not a bad price...I would buy it a...",0,0,0,0,1,0,0.0
246682,246683,B002Z05Z6C,A2KO345WG491BW,Fraulein Grey,0,0,5,2011-02-21,Wow!,My product arrived promptly in packing peanuts...,0,0,0,0,1,0,0.0
246681,246682,B002Z05Z6C,AQM475XS3T3Z7,Elizabeth Short,0,0,5,2011-04-14,LOVE this chocolate!,I recently went to Japan and although I couldn...,0,0,0,0,1,0,0.0
246678,246679,B003WSNDMY,A31QXJGLKSH2UD,LORDJOEYTX,0,0,4,2012-07-08,Exellent oil,"I haven't tried them all (yet) but, this tops ...",0,0,0,1,0,0,0.0


In [25]:
#  * WLB (Wilson Lower Bound) puanına göre sıralama: Olasılıksal bir değer arayışı içindeyiz. Güven aralığının
# alt sınırını bulmaya çalışıyoruz. Öncelikle fonksiyon yazacağız:
def wilson_lower_bound(up, total, confidence=0.95):
    if total == 0:
        return 0
    elif total < up:
        return 0
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    phat = 1.0 * up / total
    return (phat + z * z / (2 * total) - z * math.sqrt((phat * (1 - phat) + z * z / (4 * total)) / total)) / \
    (1 + z * z / total)
#şimdi de veri setimize uygulayalım: 
fine_food["WLB_Score"] = fine_food.apply(lambda x: wilson_lower_bound(x["HelpfulnessNumerator"], 
                                                                     x["HelpfulnessDenominator"]), axis=1)
fine_food.sort_values("WLB_Score", ascending=False)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,1_point,2_point,3_point,4_point,5_point,Up-Down_Sorting,Avg_Review_Point,WLB_Score
467800,467801,B0065G59F4,A1B4MIXYVIKMU2,"D. Truong ""Duke of New Mexico""",559,562,5,2011-09-02,Works great! But how does it compare to Solofi...,This Ecobrew reusable Keurig K-cup is great fo...,0,0,0,0,1,556,0.994662,9.844244e-01
96104,96105,B00859FH7O,A1B4MIXYVIKMU2,"D. Truong ""Duke of New Mexico""",559,562,5,2011-09-02,Works great! But how does it compare to Solofi...,This Ecobrew reusable Keurig K-cup is great fo...,0,0,0,0,1,556,0.994662,9.844244e-01
293135,293136,B008VQ01Y0,A1B4MIXYVIKMU2,"D. Truong ""Duke of New Mexico""",559,562,5,2011-09-02,Works great! But how does it compare to Solofi...,This Ecobrew reusable Keurig K-cup is great fo...,0,0,0,0,1,556,0.994662,9.844244e-01
210511,210512,B005FKRTO0,A1B4MIXYVIKMU2,"D. Truong ""Duke of New Mexico""",559,562,5,2011-09-02,Works great! But how does it compare to Solofi...,This Ecobrew reusable Keurig K-cup is great fo...,0,0,0,0,1,556,0.994662,9.844244e-01
385450,385451,B0051SU0OW,A1B4MIXYVIKMU2,"D. Truong ""Duke of New Mexico""",559,562,5,2011-09-02,Works great! But how does it compare to Solofi...,This Ecobrew reusable Keurig K-cup is great fo...,0,0,0,0,1,556,0.994662,9.844244e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312361,312362,B001E5E2QS,A1PJNNAFB8VO7I,"Loren Castro ""climber""",0,21,3,2008-09-22,Question,"Torani Syrup, Vanilla, 25.4-Ounce Bottles (Pac...",0,0,1,0,0,-21,0.000000,-1.173174e-17
7673,7674,B001E5DZTS,A38AAGX6FQCT7O,A. Miller,0,21,1,2010-09-23,Never Received Goat Milk,I was given a track number and told it was del...,1,0,0,0,0,-21,0.000000,-1.173174e-17
424749,424750,B001RJ7D26,A3PLZOQ7V2WFFY,Carol J. Wells,0,21,1,2011-02-28,why did I order this,I just dont understand why the date on my frui...,1,0,0,0,0,-21,0.000000,-1.173174e-17
314887,314888,B001EO7EQY,ATBUENQQVMO0J,K. Kasten,0,21,3,2009-04-25,Not Sure Yet... The FDA Still Worried About Al...,Arrived just as promised. I haven't tried the...,0,0,1,0,0,-21,0.000000,-1.173174e-17


In [26]:
# AB Testi
#   * İki örneklem ortalaması karşılaştırma
#   * İki grup oran karşılaştırma
#   * İkiden fazla grupp ortalaması karşılaştırma (ANOVA)
# Veri seti: Mobile Games A/B Testing - Cookie Cats
ab_test = pd.read_csv("../input/mobile-games-ab-testing-cookie-cats/cookie_cats.csv")

In [27]:
# veri setinde oynanan level sayını ortalamaları karşılaştırmak için kullanabiliriz. 1 gün ve 7 gün sonunda
#oyuna devam edilip edilmediğini görmek için de oranları karşılaştırabiliriz. 
# * İki örneklem ortalaması karşılaştırma: bu veri setinde iki farklı örneklem olduğu için bağımsız iki grup
# karşılaştırması için test yapacağız. 
#Adım 1: Hipotezi kur
# Ho: İki örneklemin 14 gün içinde oynadıkları seviye sayısı arasında fark bulunmamaktadır - > M1 = M2
# H1: ..... fark bulunmaktadır. -> M1 != M2
#Adım 2: Varsayım kontrolü
#Aşama 1: Normallik varsayımı (Shapiro testi)
# Ho: Normallik varsayımı sağlanmaktadır
# H1: Örnek dağılımları normal dağılım göstermemektedir 

In [28]:
from scipy.stats import shapiro
# Öncelikle, ilk grubumuz için yapalım:
test_stat, pvalue = shapiro(ab_test.loc[ab_test["version"] == "gate_30", "sum_gamerounds"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 0.0881, p-value = 0.0000


/opt/conda/lib/python3.7/site-packages/scipy/stats/morestats.py:1760: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


In [29]:
# Şimdi de ikinci grubumuz için yapalım:
test_stat, pvalue = shapiro(ab_test.loc[ab_test["version"] == "gate_40", "sum_gamerounds"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))
#iki grubumuz da normal dağılım göstermiyor. O zaman varyans homojenliğine bakmamıza gerek yok. Parametrik 
#olmayan Mann Whitney U testi kullanacağız. 

Test Stat = 0.4826, p-value = 0.0000


In [30]:
# Ho: İki örneklemin 14 gün içinde oynadıkları seviye sayısı arasında fark bulunmamaktadır - > M1 = M2
# H1: ..... fark bulunmaktadır. -> M1 != M2
# p-value değeri hesaplayıp, 0.05 altındaysa Ho'ı reddedeceğiz, değilse Ho'ı reddetmek için elimizde yeterli 
#bilgi olmadığı sonucuna varacağız. 
from scipy.stats import mannwhitneyu
test_stat, pvalue = mannwhitneyu(ab_test.loc[ab_test["version"] == "gate_30", "sum_gamerounds"],
                                 ab_test.loc[ab_test["version"] == "gate_40", "sum_gamerounds"])

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))
#test sonucumuz 0.05'in biraz üzerinde çıktı. Bu durumda Ho hipotezini reddetmek için elimizde yeterli veri 
#olmadığı sonucuna varabiliriz. Test sonucu 0.05'e çok yakın olduğu için, yeni örneklemler üzerinden gözlem 
#yapmamız ve gate_30 ile gate_40 arasında istatistiksel farklılık olmadığını tekrar test etmemiz faydalı 
#olacaktır

Test Stat = 1024331250.5000, p-value = 0.0502


In [31]:
# * İki örneklem oranları karşılaştırma: bu veri setinde iki farklı örneklem olduğu için bağımsız iki grup
# karşılaştırması için test yapacağız. 
#Adım 1: Hipotezi kur
# Ho: İki örneklemin 1 gün sonunda oyunu bırakıp bırakmama oranları arasında fark bulunmamaktadır - > p1 = p2
# H1: ..... fark bulunmaktadır. -> p1 != p2
#Adım 2: Oranlar karşılaştırılırken, Normallik veya varyans homojenliğine bakılmıyor. Proportions z-test ile
#sonuca gideceğiz: 
#Öncelikle iki grup için başarılı gözlem sayısını çıkarmamız gerekiyor: 
gate_30_success_count = ab_test.loc[ab_test["version"] == "gate_30", "retention_1"].sum()
gate_40_success_count = ab_test.loc[ab_test["version"] == "gate_40", "retention_1"].sum()
#proportions z-test uygulaması: 
from statsmodels.stats.proportion import proportions_ztest
test_stat, pvalue = proportions_ztest(count=[gate_30_success_count, gate_40_success_count],
                                      nobs=[ab_test.loc[ab_test["version"] == "gate_30", 
                                                        "retention_1"].shape[0],
                                            ab_test.loc[ab_test["version"] == "gate_40", 
                                                        "retention_1"].shape[0]])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))
#test sonucumuz 0.07 yani 0.05'in üzerinde. Bu durumda Ho hipotezini reddetmek için elimizde yeterli veri 
#olmadığı sonucuna varabiliriz. 

Test Stat = 1.7841, p-value = 0.0744


In [32]:
#Şimdi de 7 gün içerisindeki oynamaya devam etme oranlarını karşılaştıralım. Belki de 1 gün içerisinde 
#gözlemlemediğimiz fark 7 gün içerisinde oluşmuştur
#Adım 1: Hipotezi kur
# Ho: İki örneklemin 7 gün sonunda oyunu bırakıp bırakmama oranları arasında fark bulunmamaktadır - > p1 = p2
# H1: ..... fark bulunmaktadır. -> p1 != p2
#Adım 2: Oranlar karşılaştırılırken, Normallik veya varyans homojenliğine bakılmıyor. Proportions z-test ile
#sonuca gideceğiz: 
#Öncelikle iki grup için başarılı gözlem sayısını çıkarmamız gerekiyor: 
gate_30_success_count = ab_test.loc[ab_test["version"] == "gate_30", "retention_7"].sum()
gate_40_success_count = ab_test.loc[ab_test["version"] == "gate_40", "retention_7"].sum()
#proportions z-test uygulaması: 
from statsmodels.stats.proportion import proportions_ztest
test_stat, pvalue = proportions_ztest(count=[gate_30_success_count, gate_40_success_count],
                                      nobs=[ab_test.loc[ab_test["version"] == "gate_30", 
                                                        "retention_1"].shape[0],
                                            ab_test.loc[ab_test["version"] == "gate_40", 
                                                        "retention_1"].shape[0]])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))
#Şüphelendiğimiz gibi olmuş. p-value değeri bu sefer 0.05'in altında hatta sıfıra oldukça yakın bir değer.
#bu da demek oluyor ki, iki örneklem arasında 7 gün sonunda oyuna devam etme konusunda gözle görülür bir 
#farklılık yer alıyor.

Test Stat = 3.1644, p-value = 0.0016
